In [62]:
import pandas as pd

train = pd.read_csv('./data/clean_labeledTrainData.tsv', delimiter='\t')
test =  pd.read_csv('./data/clean_testData.tsv', delimiter='\t')

train.head()


print(train['review'][0])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [57]:
test.head()

,id,review
0,12311_10,naturally film main themes mortality nostalgia...
1,8348_2,movie disaster within disaster film full great...
2,5828_4,movie kids saw tonight child loved one point k...
3,7186_2,afraid dark left impression several different ...
4,12128_7,accurate depiction small time mob life filmed ...


In [74]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords # 导入停用词

def review_to_words(raw_review, remove_stop_words):
    """
    数据预处理：提取、清晰、分词
    :param raw_review:
    :return:
    """
    
    # 去除html标签
    review_text = BeautifulSoup(raw_review, "html.parser").get_text()
    # 去除非字母
    letter_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # 转化成小写字母
    words = letter_only.lower().split()
    if  remove_stop_words:
        stop_words =  set(stopwords.words('english'))
        # 去除停用词
        words = [w for w in words if not w in stop_words]
    return words


# 对train 和test分别进行上述处理
x_train = []
for review in train['review']:
    x_train.append(review)

x_test = []
for review in test['review']:
    x_test.append(review)

y_train = train['sentiment']

In [75]:
print(x_train[0])


stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

## 特征处理

特征处理 计算机无法直接处理词语，需要将文本进行向量化常见的向量化方式有一下几种
* one-hot
* 单词计数：from sklearn.feature_extraction.text import CountVectorizer 
* TF-IDF from sklearn.feature_extraction.text import TfidfVVectorizer 
* word2vec： gensim

这里先用 CountVectorizer 处理， 然后再试着用TfidfVVectorizer处理

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer =  CountVectorizer(analyzer='word',
                              tokenizer=None,
                              preprocessor=None,
                              stop_words=None,
                              max_features=5000)

x_train_features= vectorizer.fit_transform(x_train)
x_test_features = vectorizer.fit_transform(x_test)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [69]:
# 使用CountVectorizer 训练朴素贝叶斯


from sklearn.naive_bayes import MultinomialNB

print("开始训练...")
model_NB = MultinomialNB()

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) #
model_NB.fit(x_train_features, y_train) # 训练
from sklearn.cross_validation import cross_val_score
print("多项式贝叶斯10折交叉得分",
      np.mean(cross_val_score(model_NB, x_train_features, y_train, cv=10, scoring='roc_auc')))

# 效果比使用Tfidf向量化效果差很多


开始训练...
多项式贝叶斯10折交叉得分 0.916996896


In [77]:
#使用Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf  = TfidfVectorizer(min_df=2,#最小支持度
                         max_features=None,
                         strip_accents='unicode',
                         analyzer='word',
                         token_pattern=r'\w{1,}',
                         ngram_range=(1, 3),  # 二元文法模型
                         use_idf=1,
                         smooth_idf=1,
                         sublinear_tf=1,
                         stop_words = 'english')
data_all =  x_train +  x_test
len_train = len(x_train)

tfidf.fit(data_all) # 拟合
data_all = tfidf.transform(data_all) #转化

#回复成训练集和测试集

x_train = data_all[:len_train]
x_test = data_all[len_train:]

print("tf-idf 处理结束")




TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [41]:
# 使用朴素贝叶斯进行训练
from sklearn.naive_bayes import MultinomialNB 
model_NB =  MultinomialNB()
model_NB.fit(x_train, y_train) # 训练
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True) #

from sklearn.cross_validation import cross_val_score
print("多项式贝叶斯10折交叉得分",
      np.mean(cross_val_score(model_NB, x_train, y_train, cv=10, scoring='roc_auc')))




多项式贝叶斯10折交叉得分 0.9506237439999999


In [39]:
test_prediction = np.array(model_NB.predict(x_test))
print("预测结束、保存结果...")
mode_output = pd.DataFrame(data=test_prediction, columns=['sentiment'])
mode_output['id'] = test['id']
mode_output = mode_output[['id','sentiment']]
mode_output.to_csv('MultinomialNB_prediction.csv', index = False)
print("done， byebye!!!")

预测结束、保存结果...
done， byebye!!!


In [ ]:
# 使用逻辑斯谛回归
from sklearn.linear_model import LogisticRegression

# 网格搜索超参数
from sklearn.grid_search import GridSearchCV

grid_values ={'C':[30]}
model_LR = GridSearchCV(LogisticRegression(penalty = 'L2',
                                              dual = True,
                                           random_state = 0), 
                        grid_values, 
                        scoring = 'roc_auc', 
                        cv = 20)

model_LR.fit(x_train,y_train)

#20折交叉
GridSearchCV(cv=20, estimator=LogisticRegression(C=1.0, class_weight=None, dual=True,
             fit_intercept=True, intercept_scaling=1, penalty='L2', random_state=0, tol=0.0001),
        fit_params={}, iid=True, n_jobs=1,
        param_grid={'C': [30]}, pre_dispatch='2*n_jobs', refit=True,
        scoring='roc_auc', verbose=0)
#输出结果
print(model_LR.grid_scores_)
